# Model Implementation

**Security DS - Proyecto 1**

Roberto Figueroa - 18306

Gustavo Méndez - 18500

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import reduce_memory_usage
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


from sklearn import metrics, model_selection, tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, plot_confusion_matrix, accuracy_score

%matplotlib inline
pd.set_option("display.max_columns", None)

In [26]:
# Load by chunks, due to dataset size (~4 GB if unzipped)
chunk_samples = []
CHUNK_SIZE = 10 ** 6

for chunk in pd.read_csv('./datasets/preprocessed_dataset.csv', index_col=[0], chunksize=CHUNK_SIZE):
    reduced_chunk = reduce_memory_usage(chunk, verbose=False)
    chunk_samples.append(reduced_chunk)
    
df = pd.concat(chunk_samples, ignore_index=True)
del chunk_samples

In [27]:
df.head()

,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_SCALE_OUT,TCP_WIN_SCALE_IN,TOTAL_FLOWS_EXP,tcp,udp,SRC_PKT_PRECEDENCE,DST_PKT_PRECEDENCE,URG,ACK,PSH,RST,SYN,FIN,LABEL
0,1618194298,-0.821289,-0.116516,0.700684,1618194298,-0.800293,-0.744141,-0.804688,-0.743652,-1.385742,-0.768555,-0.816895,0.512207,1.0,0.0,5.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.447266
1,1618185270,-0.822754,-0.014404,0.324707,1618185270,-0.838867,-0.744141,-0.837402,-0.743652,-1.385742,-0.768555,-0.816895,0.503906,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.447266
2,1618225083,-0.822754,-0.570801,-1.906250,1618225083,-0.838867,-0.744141,-0.837402,-0.743652,-1.385742,-0.768555,-0.816895,0.582031,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.447266
3,1618219151,-0.817871,-0.309570,0.797363,1618219152,-0.800293,-0.744141,-0.804688,-0.743652,-1.385742,-0.768555,-0.816895,0.559082,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.447266
4,1618197175,-0.811523,-0.576660,-2.744141,1618197176,-0.838867,-0.744141,-0.837402,-0.743652,-1.385742,-0.768555,-0.816895,0.515137,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.447266


### Split data

In [28]:
y = df['LABEL']
X = df.drop('LABEL', axis=1, inplace=False)

In [29]:
X_train, X_test_total, y_train, y_test_total = train_test_split(X, y, train_size=0.55, random_state=42)

In [30]:
X_test, X_val, y_test, y_val = train_test_split(X_test_total, y_test_total, test_size=0.33, random_state=97)

In [32]:
print("Total observaciones: ", len(X))
print("Observaciones entrenamiento: ", len(X_train))
print("Observaciones de validacion: ", len(X_val))
print("Observaciones de pruebas: ", len(X_test))

Total observaciones:  733922
Observaciones entrenamiento:  403657
Observaciones de validacion:  108988
Observaciones de pruebas:  221277


### Create model

In [34]:
model_RF = RandomForestClassifier(n_estimators = 100, random_state = 42)
model_RF.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

#### Train with validation data

In [ ]:
val_predictions = model.predict(X_val)

In [ ]:
conf_matrix = confusion_matrix(y_val, val_predictions)
accu = accuracy_score(y_val, val_predictions)
_recall_score = recall_score(y_val, val_predictions)
_precision_score = precision_score(y_val, val_predictions)
_f1_score = f1_score(y_val, val_predictions)

print("\n Confusion Matrix", conf_matrix)
print("\n Accuracy", accu)
print("\n Precision", _precision_score)
print("\n Recall", _recall_score)
print("\n F1 Score", _f1_score)

In [ ]:
target_names = [0, 1, 2, 3]
print(metrics.classification_report(y_val, val_predictions, target_names=target_names))

In [ ]:
sns.heatmap(conf_matrix, annot=True, fmt=".4g");

### Train model

### Metrics

### K-fold validation